L'objet de cet exercice consiste à créer un modèle de reconnaissance d'image capable de prédire s'il y a un père noël sur l'image. Le modèle devra être le plus précis en suivant une méthode qui minimise la taille des données pour un maximum de précision.

# Générateur d'images

Avant d'utiliser les images d'une base de données, il faut souvent traiter en amont les images. En deep learning, le réseau de neurones doit s'entraîner sur des lots d'images de la base de données. La classe ImageDataGenerator permet de générer des lots d'images transformées pour entraîner votre réseau. Dans ce test vous devrez vous appuyer sur la documentation cliquez ici.

- Import ImageDataGenerator du sous-module keras.preprocessing.image
- Créer train_datagen une instance de ImageDataGenerator avec pour paramètres :
    rescale = 1./255, <br>
    shear_range = 0.2, <br>
    zoom_range = 0.2, <br>
    rotation_range = 40, <br>
    width_shift_range = 0.2, <br>
    height_shift_range = 0.2, <br>
    horizontal_flip = True, <br>
    fill_mode = 'nearest' <br>
- Créer test_datagen, une instance de ImageDataGenerator avec comme unique paramètre rescale = 1./255

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

# Base de données et importation des données

Deux dossiers s'intitulant images et images_test contiennent chacun deux sous-dossiers d'images. santa qui ne contient que des images de père Noël, et not_santa  qui ne contient que des images aléatoires d'objets et de personnes qui ne sont pas des pères Noël. <br>
Pour importer et transformer les images avec la classe ImageDataGenerator, on utilise la méthode flow_from_directory qui prend en argument le chemin donnant accès aux dossiers. Elle détecte automatiquement les images et les classes dans deux catégories différentes car les images sont dans deux dossier différents. <br>
Pour plus d'information sur flow_from_directory, cliquez ici.

- Pour importer vos images, lancer le code suivant.

In [ ]:
#Lancer le code suivant

#On prend des lots de 32

training_set = train_datagen.flow_from_directory('images',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('images_test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')


# Affichage de la donnée "Augmentée"

Observons le réultat d'une augmentation de données

- Exécuter la cellule suivante pour observer des données augmentées

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

batches_real = test_datagen.flow_from_directory('images', target_size = (512, 512), batch_size = 16, class_mode = 'categorical')
batches_augmented = train_datagen.flow_from_directory('images', target_size = (512, 512), batch_size = 16, class_mode = 'catgorical')

x_batch_augmented, y_batch_augmented = next(batches_augmented)
x_batch_real, y_batch_real = next(batches_real)

for i in range(16):
    image_augmented = x_batch_augmented[i]
    image_real = x_batch_real[i]
    
    title_add_on = 'random image'
    if y_batch_augmented[i][1]: title_add_on = "santa"
        
        plt.subplot(221)
        plt.imshow(image_real)
        plt.title("original" + title_add_on)
        
        plt.subplot(222)
        plt.imshow(image_augmented)
        plt.title("augmented" + title_add_on)
        
        plt.show()

- Quelles sont les transformations apportées sur la data augmentée ? <br>
- Selon vous, en quoi l'augmentation de données est-elle utile pour l'entrainement d'un réseau de neurones ?

In [ ]:
'''
Les transformations apportées par la data augmentée sont des combinaisons des transformations suivantes :
- shear_range = 0.2 : cisaillement de l'image,
- zoom_range = 0.2 : un zoom allant jusqu'à 20% de l'image,
- rotation_range = 40 : des rotations pouvant aller jusqu'à 40°,
- width_shift_range = 0.2 : translation de l'image en horizontal pouvant aller de + ou - 20% de la largeur de l'image,
- height_shift_range = 0.2 : translation de l'image en vertical pouvant aller de + ou - 20% de la hauteur de l'image,
- horizontal_flip = True : retournement de l'image horizontalement,
- fill_mode = 'nearest' : les pixels en dehors de l'image originale sont remplis de la manière aaaaaaaaa|abcd|dddddddd



Si on entraine un réseau de neurones sur trop peu d'images, le modèle pourrait avoir tendance à faire de l'overfitting
(sur-apprendre à partir de l'échantillon d'entrainement sans être capable d'effectuer des prédictions pertinentes
sur de nouvelles images). Pour résoudre ce problème, une solution est d'augmenter la taille du dataset. Seulement,
labelliser tout un set d'images peut être très chronophage. L'augmentation de données permet d'éviter ce problème et
de générer de nouvelles images labellisées à partir de celles déjà disponibles. 
L'augmentation de données permettra donc d'éviter l'overfitting notamment.
'''

# Création du réseau de neurones

Pour classer ces images, vous pouvez vous appuyer sur le réseau de neurones suivant : 

La sortie de la méthode .summary() de votre modèle devra ressembler à ceci :

- Implémenter votre modèle dans une instance nommée classifier.
- Entrainer votre modèle jusqu'à atteindre une précision de validation val_accuracy supérieure à 0.85 en choisissant judicieusement les paramètres d'entrainement. <br>

NB : training_set est un générateur, il faut utiliser la méthode fit_generator là où vous avez l'habitude d'utiliser la méthode fit.

In [ ]:
classifier = Sequential()

conv2d_5 = conv2D(filters = 30,
                  kernel_size = (5, 5),
                  padding = 'valid',
                  input_shape = (64, 64, 3),
                  activation = 'relu')
max_pooling2d_5 = MaxPooling2D(pool_size = (2, 2))
conv2d_6 = Conv2D(filters = 16,
                   kernel_size = (3, 3),
                   padding = 'valid',
                   activation = 'relu')
max_pooling2d_6 = MaxPooling2D(pool_size = (2, 2))
flatten_3 = Flatten()
dense_5 = Dense(units = 128,
                activation = 'relu')
dense_6 = Dense(units = 10,
                activation = 'softmax')

classifier.add(conv2d_5)
classifier.add(max_pooling2d_5)
classifier.add(conv2d_6)
classifier.add(max_pooling2d_6)
classifier.add(flatten_3)
classifier.add(dense_5)
classifier.add(dense_6)

In [ ]:
from keras.optimizers import Adam, Adamax, SGD

classifier.compile(optimizer = Adam(learning_rate = 1e-3), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#categorical_crossentropy

In [ ]:
classifier.summary()

In [ ]:
nb_train = training_set.n #nb d'images
nb_test = test_set.n

history = classifier.fit_generator(generator = training_set,
                              epochs = 5,
                              steps_per_epoch = nb_train//32,
                              validation_data = test_set,
                              validation_steps = nb_test//32)

In [ ]:
plt.figure(figsize = (12, 4))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model acc by epoch')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()

# Test du modèle

- Exécuter le code suivant pour observer la probabilité que l'on obtient pour une image de Thor.

In [ ]:
from keras.preprocessing import image
import matplotlib.image as mpimg
import numpy as np

txt = 'thor.jpg' # Préciser le chemin local
text_image = image.load_img(txt, target_size = (64, 64))
test_image = image.img_to_array(test_image)/255
test_image = np.expand_dims(test_image, axis = 0)

proba = round(100*classifier.predict_proba(test_image)[0][1], 2)
if proba < 50:
    santa_or_not = 'Santa'
img = mpimg.imread(txt)
plt.axis('off')
plt.text(-10, -15, santa_or_not+': '+str(proba)+'%', color = (1, 0, 0), fontsize = 20, fontweight = 'extra bold')
imgplot = plt.imshow(img)

Réitérer le test avec les photos suivantes :
- chien.jpg
- chat.jpg
- selfie.jpg
- santa_rock.jpg

In [ ]:
images = ['chien.jpg', 'chat.jpg', 'selfie.jpg', 'santa_rock.jpg']
j = 1

for img in images:
    text_image = image.load_img(img, target_size = (64, 64))
    test_image = image.img_to_array(test_image)/255
    test_image = np.expand_dims(test_image, axis = 0)

    proba = round(100*classifier.predict_proba(test_image)[0][1], 2)
    if proba < 50:
        santa_or_not = 'Santa'
    img = mpimg.imread(img)
    plt.subplot(2, 2, j)
    j = j+1
    plt.axis('off')
    plt.text(-10, -15, santa_or_not+': '+str(proba)+'%', color = (1, 0, 0), fontsize = 20, fontweight = 'extra bold')
    imgplot = plt.imshow(img)